A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente. O EV/EBIT compara o Valor da Empresa (Enterprise Value, EV = Capitalização de mercado + Dívida total - Caixa e equivalentes de caixa) ao seu Lucro Antes de Juros e Impostos (Earnings Before Interest and Taxes, EBIT), o EV leva em conta o valor de mercado total da empresa, adicionando as dívidas e subtraindo o caixa, oferecendo assim, uma visão mais abrangente do valor da empresa do que simplesmente seu valor de mercado. O EBIT é uma medida da capacidade de geração de lucro operacional da empresa, excluindo os efeitos das decisões de estrutura de capital e impostos. Um EV/EBIT menor sugere que o preço da ação da empresa pode estar baixo em comparação com a quantidade de lucro operacional que está gerando, indicando que a empresa está subavaliada pelo mercado. No entanto, é importante analisar o EV/EBIT dentro do contexto de cada setor, já que diferentes indústrias têm padrões de avaliação distintos.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render. O ROIC é um dos indicadores financeiros mais importantes para analisar a eficiência operacional de uma empresa. Para calcular o ROIC, divide-se o lucro operacional líquido de impostos pelo capital investido total, que inclui dívidas e patrimônio líquido. Um ROIC alto é geralmente interpretado como um sinal de que a empresa está utilizando seu capital de maneira eficaz para gerar lucros.

ROIC utilizada por Greenblatt é diferente da fórmula tradicional

ROIC = EBIT / (Capital Circulante Líquido + Ativos Fixos Líquidos)

Onde o Capital circulante Líquido é um proxy do Capital de Giro da Empresa (Ativo Circulante – Passivo Circulante), os Ativos Fixos Líquidos é proxy dos ativos permanentes (imobilizado e intangíveis, utilizados para gerar retorno no longo prazo. A Taxa de Retorno de Lucros para uma empresa é a fórmula tradicional: Taxa de Retorno = Lucro Operacional / Valor da Empresa

Estes dois indicadores são colocados cada um em um ranking, da melhor (rank 1) para a pior empresa. Depois somamos os números das duas colunas, quanto menor a soma, melhor.

Para os princípios de Greenblatt, empresas pertencentes aos setores de utilidades públicas, bancos e  seguradoras devem ser excluídas da seleção. 

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
from IPython.display import display, HTML


def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))


def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None

def setor(setor:int):

    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    content = requests.get(url, headers=hdr)
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]
    return list(df['Papel'])



dados do site https://www.fundamentus.com.br/resultado.php

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
data_funds = pd.DataFrame(funda)
data_funds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [4]:
earning_yield = 'EV/EBIT'  #Quanto menor melhor
return_on_capital = 'ROIC' #ROIC – Quanto maior melhor

### filtros e ranking

- Empresa precisa ter ROIC positivo
- Empresa precisa ter Lucro Operacional positivo.
- Volume de negociação diário medio maior que R$ 1000000.000,00 nos últimos 2 meses.

ROIC – Quanto maior melhor

EV/EBIT – Quanto menor melhor

In [5]:
funds = data_funds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]
funds =  funds[funds['Liq.2meses'] > 1000000]     #Volume diário médio negociado nos últimos 2 meses

""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1

### lista ranqueada

In [6]:
funds.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a',
       'Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final'],
      dtype='object')

In [7]:
funds[['Papel','EV/EBITDA', 'ROIC', 'Liq.2meses','Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final']].head(20)

,Papel,EV/EBITDA,ROIC,Liq.2meses,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,0.50,75.32,8.227410e+07,1.0,1.0,2.0
2,WIZC3,1.38,31.37,2.068400e+06,2.0,7.0,9.0
3,PLPL3,2.96,44.22,1.400820e+07,7.0,2.0,9.0
4,CMIN3,3.62,30.62,3.372700e+07,23.0,8.0,31.0
5,PETR4,2.79,20.22,1.161370e+09,13.0,21.0,34.0
6,LREN3,2.52,19.09,2.525490e+08,11.0,25.0,36.0
7,KEPL3,3.98,26.04,1.090830e+07,24.0,12.0,36.0
8,SYNE3,1.47,16.74,3.379830e+06,3.0,33.0,36.0
9,QUAL3,1.65,17.15,5.430820e+06,6.0,32.0,38.0
10,PETR3,2.91,20.22,3.664840e+08,18.0,21.0,39.0


retira banco, seguradora, utilidade publica (agua e luz - [util-b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-utilidade-publica-util-composicao-da-carteira.htm))


In [8]:
bancos = setor(20)
print(*bancos)
seguros = setor(31)
print(*seguros)
util_agua = setor(2)
util_luz = setor(14)
util=util_agua+util_luz
print(*util)


RPAD5 RPAD3 RPAD6 BGIP4 BGIP3 BAZA3 PINE4 PINE3 BNBR3 BBAS3 BMEB3 BRSR6 ABCB4 BMEB4 BRSR3 BMGB4 BRSR5 BBDC3 BBDC4 BEES3 BEES4 ITSA4 SANB3 ITSA3 BRBI11 SANB11 BPAC5 ITUB3 SANB4 ITUB4 BPAC11 BMIN4 BPAN4 BMIN3 BSLI4 MERC4 BSLI3 BPAC3
IRBR3 WIZC3 BBSE3 PSSA3 CXSE3
AMBP3 SAPR4 SAPR11 SAPR3 CSMG3 SBSP3 ORVR3 GPAR3 LIGT3 CEED3 GEPA3 GEPA4 RNEW4 RNEW3 RNEW11 CMIG4 COCE5 COCE3 CLSC3 ISAE4 CLSC4 EQMA3B CMIG3 CEEB3 EQPA3 CBEE3 ISAE3 ENGI4 NEOE3 CEBR5 CEBR3 EGIE3 ENGI11 EQPA5 CEBR6 TAEE3 CPFE3 TAEE11 TAEE4 ALUP4 ALUP11 ALUP3 REDE3 ELET3 CPLE3 ELET6 CPLE6 EKTR4 ENGI3 EMAE4 AFLT3 CPLE5 ENMT4 ENMT3 SRNA3 ELET5 EQTL3 LIPR3 ENEV3 AURE3


#### sem bancos

In [9]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(bancos)],10
)
          

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,37.40,9.14,1.72,0.686,4.12,0.505,7.08,0.84,-3.23,0.51,0.50,81.86,7.63,1.15,75.32,18.82,82274100.0,1.405080e+10,0.00,18.52,1.0,1.0,2.0,PSSA
WIZC3,6.06,5.39,1.62,0.767,4.06,0.388,-6.22,1.51,-1.36,1.65,1.38,50.82,24.55,0.77,31.37,29.99,2068400.0,5.991930e+08,0.57,13.02,2.0,7.0,9.0,WIZC
PLPL3,10.74,6.45,2.60,0.857,9.37,0.887,1.32,3.21,3.77,3.05,2.96,26.71,14.58,4.42,44.22,40.32,14008200.0,8.414080e+08,0.70,28.06,7.0,2.0,9.0,PLPL
CMIN3,5.44,7.71,3.34,1.737,14.30,0.866,4.10,5.42,-3.35,4.38,3.62,32.04,22.53,1.78,30.62,43.27,33727000.0,8.945330e+09,0.98,-2.60,23.0,8.0,31.0,CMIN
PETR4,34.62,12.19,1.22,0.909,23.01,0.397,-7.49,2.12,-0.72,3.67,2.79,42.87,7.54,0.69,20.22,10.00,1161370000.0,3.660060e+11,1.02,13.92,13.0,21.0,34.0,PETR
LREN3,11.86,10.50,1.17,0.870,5.08,0.617,2.69,4.17,4.61,3.56,2.52,20.87,8.29,1.61,19.09,11.11,252549000.0,1.077300e+10,0.09,16.83,11.0,25.0,36.0,LREN
KEPL3,7.58,6.84,1.84,0.848,11.13,0.855,2.58,4.88,6.28,4.55,3.98,17.37,12.39,1.98,26.04,26.89,10908300.0,7.407810e+08,0.44,21.60,24.0,12.0,36.0,KEPL
SYNE3,4.70,1.46,0.46,0.514,139.39,0.189,0.67,1.38,-5.20,1.70,1.47,37.24,44.92,11.34,16.74,31.25,3379830.0,1.573570e+09,0.54,21.71,3.0,33.0,36.0,SYNE
QUAL3,1.82,78.57,0.40,0.327,0.00,0.118,3.35,1.01,-0.37,2.94,1.65,32.39,0.95,1.10,17.15,0.51,5430820.0,1.282270e+09,1.47,-6.55,6.0,32.0,38.0,QUAL
ASAI3,7.07,12.43,1.82,0.130,1.31,0.210,70.29,1.79,-0.40,3.83,2.93,7.24,1.04,1.01,18.89,14.63,133041000.0,5.255000e+09,3.15,20.87,17.0,27.0,44.0,ASAI


#### sem seguradora

In [10]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(seguros)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,10.74,6.45,2.60,0.857,9.37,0.887,1.32,3.21,3.77,3.05,2.96,26.71,14.58,4.42,44.22,40.32,14008200.0,8.414080e+08,0.70,28.06,7.0,2.0,9.0,PLPL
CMIN3,5.44,7.71,3.34,1.737,14.30,0.866,4.10,5.42,-3.35,4.38,3.62,32.04,22.53,1.78,30.62,43.27,33727000.0,8.945330e+09,0.98,-2.60,23.0,8.0,31.0,CMIN
PETR4,34.62,12.19,1.22,0.909,23.01,0.397,-7.49,2.12,-0.72,3.67,2.79,42.87,7.54,0.69,20.22,10.00,1161370000.0,3.660060e+11,1.02,13.92,13.0,21.0,34.0,PETR
LREN3,11.86,10.50,1.17,0.870,5.08,0.617,2.69,4.17,4.61,3.56,2.52,20.87,8.29,1.61,19.09,11.11,252549000.0,1.077300e+10,0.09,16.83,11.0,25.0,36.0,LREN
KEPL3,7.58,6.84,1.84,0.848,11.13,0.855,2.58,4.88,6.28,4.55,3.98,17.37,12.39,1.98,26.04,26.89,10908300.0,7.407810e+08,0.44,21.60,24.0,12.0,36.0,KEPL
SYNE3,4.70,1.46,0.46,0.514,139.39,0.189,0.67,1.38,-5.20,1.70,1.47,37.24,44.92,11.34,16.74,31.25,3379830.0,1.573570e+09,0.54,21.71,3.0,33.0,36.0,SYNE
QUAL3,1.82,78.57,0.40,0.327,0.00,0.118,3.35,1.01,-0.37,2.94,1.65,32.39,0.95,1.10,17.15,0.51,5430820.0,1.282270e+09,1.47,-6.55,6.0,32.0,38.0,QUAL
ASAI3,7.07,12.43,1.82,0.130,1.31,0.210,70.29,1.79,-0.40,3.83,2.93,7.24,1.04,1.01,18.89,14.63,133041000.0,5.255000e+09,3.15,20.87,17.0,27.0,44.0,ASAI
LEVE3,29.28,6.79,3.58,0.932,8.07,1.213,3.50,4.92,-73.12,5.58,4.95,18.93,13.74,2.16,33.91,52.72,6565200.0,1.109300e+09,0.92,16.79,40.0,5.0,45.0,LEVE
POMO3,5.50,5.21,1.55,0.727,10.73,0.662,2.42,4.47,20.78,5.29,4.73,16.28,14.22,1.84,20.99,29.80,2336020.0,4.026610e+09,0.81,27.04,28.0,19.0,47.0,POMO


#### sem utilidade publica

In [11]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(util)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,37.40,9.14,1.72,0.686,4.12,0.505,7.08,0.84,-3.23,0.51,0.50,81.86,7.63,1.15,75.32,18.82,82274100.0,1.405080e+10,0.00,18.52,1.0,1.0,2.0,PSSA
WIZC3,6.06,5.39,1.62,0.767,4.06,0.388,-6.22,1.51,-1.36,1.65,1.38,50.82,24.55,0.77,31.37,29.99,2068400.0,5.991930e+08,0.57,13.02,2.0,7.0,9.0,WIZC
PLPL3,10.74,6.45,2.60,0.857,9.37,0.887,1.32,3.21,3.77,3.05,2.96,26.71,14.58,4.42,44.22,40.32,14008200.0,8.414080e+08,0.70,28.06,7.0,2.0,9.0,PLPL
CMIN3,5.44,7.71,3.34,1.737,14.30,0.866,4.10,5.42,-3.35,4.38,3.62,32.04,22.53,1.78,30.62,43.27,33727000.0,8.945330e+09,0.98,-2.60,23.0,8.0,31.0,CMIN
PETR4,34.62,12.19,1.22,0.909,23.01,0.397,-7.49,2.12,-0.72,3.67,2.79,42.87,7.54,0.69,20.22,10.00,1161370000.0,3.660060e+11,1.02,13.92,13.0,21.0,34.0,PETR
LREN3,11.86,10.50,1.17,0.870,5.08,0.617,2.69,4.17,4.61,3.56,2.52,20.87,8.29,1.61,19.09,11.11,252549000.0,1.077300e+10,0.09,16.83,11.0,25.0,36.0,LREN
KEPL3,7.58,6.84,1.84,0.848,11.13,0.855,2.58,4.88,6.28,4.55,3.98,17.37,12.39,1.98,26.04,26.89,10908300.0,7.407810e+08,0.44,21.60,24.0,12.0,36.0,KEPL
SYNE3,4.70,1.46,0.46,0.514,139.39,0.189,0.67,1.38,-5.20,1.70,1.47,37.24,44.92,11.34,16.74,31.25,3379830.0,1.573570e+09,0.54,21.71,3.0,33.0,36.0,SYNE
QUAL3,1.82,78.57,0.40,0.327,0.00,0.118,3.35,1.01,-0.37,2.94,1.65,32.39,0.95,1.10,17.15,0.51,5430820.0,1.282270e+09,1.47,-6.55,6.0,32.0,38.0,QUAL
ASAI3,7.07,12.43,1.82,0.130,1.31,0.210,70.29,1.79,-0.40,3.83,2.93,7.24,1.04,1.01,18.89,14.63,133041000.0,5.255000e+09,3.15,20.87,17.0,27.0,44.0,ASAI


#### sem as 3 classes

In [12]:
b_print(
    funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,10.74,6.45,2.60,0.857,9.37,0.887,1.32,3.21,3.77,3.05,2.96,26.71,14.58,4.42,44.22,40.32,14008200.0,8.414080e+08,0.70,28.06,7.0,2.0,9.0,PLPL
CMIN3,5.44,7.71,3.34,1.737,14.30,0.866,4.10,5.42,-3.35,4.38,3.62,32.04,22.53,1.78,30.62,43.27,33727000.0,8.945330e+09,0.98,-2.60,23.0,8.0,31.0,CMIN
PETR4,34.62,12.19,1.22,0.909,23.01,0.397,-7.49,2.12,-0.72,3.67,2.79,42.87,7.54,0.69,20.22,10.00,1161370000.0,3.660060e+11,1.02,13.92,13.0,21.0,34.0,PETR
LREN3,11.86,10.50,1.17,0.870,5.08,0.617,2.69,4.17,4.61,3.56,2.52,20.87,8.29,1.61,19.09,11.11,252549000.0,1.077300e+10,0.09,16.83,11.0,25.0,36.0,LREN
KEPL3,7.58,6.84,1.84,0.848,11.13,0.855,2.58,4.88,6.28,4.55,3.98,17.37,12.39,1.98,26.04,26.89,10908300.0,7.407810e+08,0.44,21.60,24.0,12.0,36.0,KEPL
SYNE3,4.70,1.46,0.46,0.514,139.39,0.189,0.67,1.38,-5.20,1.70,1.47,37.24,44.92,11.34,16.74,31.25,3379830.0,1.573570e+09,0.54,21.71,3.0,33.0,36.0,SYNE
QUAL3,1.82,78.57,0.40,0.327,0.00,0.118,3.35,1.01,-0.37,2.94,1.65,32.39,0.95,1.10,17.15,0.51,5430820.0,1.282270e+09,1.47,-6.55,6.0,32.0,38.0,QUAL
ASAI3,7.07,12.43,1.82,0.130,1.31,0.210,70.29,1.79,-0.40,3.83,2.93,7.24,1.04,1.01,18.89,14.63,133041000.0,5.255000e+09,3.15,20.87,17.0,27.0,44.0,ASAI
LEVE3,29.28,6.79,3.58,0.932,8.07,1.213,3.50,4.92,-73.12,5.58,4.95,18.93,13.74,2.16,33.91,52.72,6565200.0,1.109300e+09,0.92,16.79,40.0,5.0,45.0,LEVE
POMO3,5.50,5.21,1.55,0.727,10.73,0.662,2.42,4.47,20.78,5.29,4.73,16.28,14.22,1.84,20.99,29.80,2336020.0,4.026610e+09,0.81,27.04,28.0,19.0,47.0,POMO


outros filtros

In [13]:
funds2 =  funds[funds['P/L'] > 0]                # sem prejuizo atual
funds2 =  funds2[(funds['P/L'] < 30)]           # não estar excessivamente cara
funds2 =  funds2[funds['Dív.Brut/ Patrim.'] < 3.5]  # endividamento sob controle
funds2 =  funds2[funds['Cresc. Rec.5a'] > 0]    # crescimento nos ultimos 5 anos

In [14]:
b_print(
    funds2[(~funds2['Papel'].isin(bancos))&(~funds2['Papel'].isin(seguros))&(~funds2['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,10.74,6.45,2.60,0.857,9.37,0.887,1.32,3.21,3.77,3.05,2.96,26.71,14.58,4.42,44.22,40.32,14008200.0,8.414080e+08,0.70,28.06,7.0,2.0,9.0,PLPL
PETR4,34.62,12.19,1.22,0.909,23.01,0.397,-7.49,2.12,-0.72,3.67,2.79,42.87,7.54,0.69,20.22,10.00,1161370000.0,3.660060e+11,1.02,13.92,13.0,21.0,34.0,PETR
LREN3,11.86,10.50,1.17,0.870,5.08,0.617,2.69,4.17,4.61,3.56,2.52,20.87,8.29,1.61,19.09,11.11,252549000.0,1.077300e+10,0.09,16.83,11.0,25.0,36.0,LREN
KEPL3,7.58,6.84,1.84,0.848,11.13,0.855,2.58,4.88,6.28,4.55,3.98,17.37,12.39,1.98,26.04,26.89,10908300.0,7.407810e+08,0.44,21.60,24.0,12.0,36.0,KEPL
SYNE3,4.70,1.46,0.46,0.514,139.39,0.189,0.67,1.38,-5.20,1.70,1.47,37.24,44.92,11.34,16.74,31.25,3379830.0,1.573570e+09,0.54,21.71,3.0,33.0,36.0,SYNE
ASAI3,7.07,12.43,1.82,0.130,1.31,0.210,70.29,1.79,-0.40,3.83,2.93,7.24,1.04,1.01,18.89,14.63,133041000.0,5.255000e+09,3.15,20.87,17.0,27.0,44.0,ASAI
LEVE3,29.28,6.79,3.58,0.932,8.07,1.213,3.50,4.92,-73.12,5.58,4.95,18.93,13.74,2.16,33.91,52.72,6565200.0,1.109300e+09,0.92,16.79,40.0,5.0,45.0,LEVE
POMO3,5.50,5.21,1.55,0.727,10.73,0.662,2.42,4.47,20.78,5.29,4.73,16.28,14.22,1.84,20.99,29.80,2336020.0,4.026610e+09,0.81,27.04,28.0,19.0,47.0,POMO
VLID3,26.85,6.22,1.30,0.992,6.62,0.778,2.26,5.16,5.69,5.29,4.21,19.22,15.91,2.87,20.16,20.92,8154540.0,1.689020e+09,0.36,1.64,28.0,23.0,51.0,VLID
CEAB3,10.42,7.10,0.97,0.421,3.31,0.319,2.81,3.44,-1.83,3.36,2.44,12.23,5.93,1.30,14.97,13.68,38629900.0,3.308480e+09,0.45,16.37,9.0,46.0,55.0,CEAB


retirar BDR

In [15]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds[funds['Papel'].str.contains(padrao)]


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
23,AURA33,29.04,-42.34,4.57,1.958,2.72,0.943,7.38,5.29,-2.18,...,1.55,27.04,-10.79,5482350.0,1.380600e+09,1.99,13.82,59.0,11.0,70.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.

In [16]:
data_funds.loc['INEP4']

Cotação              1.360000e+00
P/L                  1.200000e-01
P/VP                -4.000000e-02
PSR                  1.405600e+01
Div.Yield            0.000000e+00
P/Ativo              5.900000e-02
P/Cap.Giro          -5.000000e-02
P/EBIT              -9.600000e-01
P/Ativ Circ.Liq     -3.000000e-02
EV/EBIT             -1.553000e+01
EV/EBITDA           -1.735000e+01
Mrg Ebit            -1.461870e+03
Mrg. Líq.            1.181240e+04
Liq. Corr.           8.000000e-02
ROIC                -6.530000e+00
ROE                 -3.690000e+01
Liq.2meses           6.668860e+04
Patrim. Líq         -1.377160e+09
Dív.Brut/ Patrim.   -6.600000e-01
Cresc. Rec.5a       -9.090000e+00
Name: INEP4, dtype: float64